In [1]:
import os
os.getcwd()
%cd ..

/Users/Hendrik/Documents/leomed_mimic/mimic


In [2]:
import torch

In [4]:
from pathlib import Path
import pandas as pd
import argparse

from tensorflow.python.summary.summary_iterator import summary_iterator
import json
import os

In [ ]:
from mimic.utils.filehandling import get_config_path, expand_paths
from mimic.notebooks.utils.boilerplate import test_clfs, test_clf, test_dummy, test_model
from mimic.utils.flags import parser
from mimic.networks.CheXNet import CheXNet
import numpy as np
from IPython.display import display
from mimic.dataio.MimicDataset import Mimic
from mimic.utils.experiment import MimicExperiment
from mimic.utils import utils

In [40]:
from pathlib import Path

import pandas as pd
import os
pd.set_option('display.max_rows', 50)
pd.options.display.max_columns = None
experiments_dataframe = pd.read_csv(Path(os.path.abspath("trained_clf_exploration.ipynb")).parent / 'experiments_dataframe.csv')
lr_eval_columns = [col for col in experiments_dataframe.columns if col.startswith('lr_eval_')]
gen_eval_columns = [col for col in experiments_dataframe.columns if col.startswith('gen_eval_')]
lhoods_columns = [col for col in experiments_dataframe.columns if col.startswith('lhoods_')]
prd_scores_columns = [col for col in experiments_dataframe.columns if col.startswith('prd_scores_')]

print(lr_eval_columns)
print(gen_eval_columns)
print(lhoods_columns)
print(prd_scores_columns)
eval_columns = [*lr_eval_columns, *gen_eval_columns, *lhoods_columns, *prd_scores_columns]
print(eval_columns)

['lr_eval_Lung Opacity_PA', 'lr_eval_Lung Opacity_Lateral', 'lr_eval_Lung Opacity_text', 'lr_eval_Lung Opacity_Lateral_PA', 'lr_eval_Lung Opacity_PA_text', 'lr_eval_Lung Opacity_Lateral_text', 'lr_eval_Lung Opacity_Lateral_PA_text', 'lr_eval_Pleural Effusion_PA', 'lr_eval_Pleural Effusion_Lateral', 'lr_eval_Pleural Effusion_text', 'lr_eval_Pleural Effusion_Lateral_PA', 'lr_eval_Pleural Effusion_PA_text', 'lr_eval_Pleural Effusion_Lateral_text', 'lr_eval_Pleural Effusion_Lateral_PA_text', 'lr_eval_Support Devices_PA', 'lr_eval_Support Devices_Lateral', 'lr_eval_Support Devices_text', 'lr_eval_Support Devices_Lateral_PA', 'lr_eval_Support Devices_PA_text', 'lr_eval_Support Devices_Lateral_text', 'lr_eval_Support Devices_Lateral_PA_text']
['gen_eval_cond_Lung Opacity_text_text', 'gen_eval_cond_Pleural Effusion_text_text', 'gen_eval_cond_Support Devices_text_text', 'gen_eval_random_Lung Opacity', 'gen_eval_random_Pleural Effusion', 'gen_eval_random_Support Devices', 'gen_eval_cond_Lung Opa

In [41]:
print(experiments_dataframe.columns)

Index(['DIM_img', 'DIM_text', 'alpha_modalities', 'batch_size', 'beta',
       'beta_1', 'beta_2', 'beta_content', 'beta_m1_style', 'beta_m2_style',
       ...
       'gen_eval_cond_Support Devices_Lateral_PA_text',
       'gen_eval_cond_Support Devices_PA_text_PA',
       'gen_eval_cond_Support Devices_PA_text_Lateral',
       'gen_eval_cond_Support Devices_PA_text_text',
       'gen_eval_cond_Support Devices_Lateral_text_PA',
       'gen_eval_cond_Support Devices_Lateral_text_Lateral',
       'gen_eval_cond_Support Devices_Lateral_text_text',
       'gen_eval_cond_Support Devices_Lateral_PA_text_PA',
       'gen_eval_cond_Support Devices_Lateral_PA_text_Lateral',
       'gen_eval_cond_Support Devices_Lateral_PA_text_text'],
      dtype='object', length=213)


,DIM_img,DIM_text,alpha_modalities,batch_size,beta,beta_1,beta_2,beta_content,beta_m1_style,beta_m2_style,...,gen_eval_cond_Support Devices_Lateral_PA_text,gen_eval_cond_Support Devices_PA_text_PA,gen_eval_cond_Support Devices_PA_text_Lateral,gen_eval_cond_Support Devices_PA_text_text,gen_eval_cond_Support Devices_Lateral_text_PA,gen_eval_cond_Support Devices_Lateral_text_Lateral,gen_eval_cond_Support Devices_Lateral_text_text,gen_eval_cond_Support Devices_Lateral_PA_text_PA,gen_eval_cond_Support Devices_Lateral_PA_text_Lateral,gen_eval_cond_Support Devices_Lateral_PA_text_text
0,64,128,"[0.25, 0.25, 0.25, 0.25]",256,1.0,0.9,0.999,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,64,128,"[0.25, 0.25, 0.25, 0.25]",180,1.0,0.9,0.999,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,64,128,"[0.25, 0.25, 0.25, 0.25]",100,1.0,0.9,0.999,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,64,128,"[0.25, 0.25, 0.25, 0.25]",100,1.0,0.9,0.999,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,64,128,"[0.25, 0.25, 0.25, 0.25]",100,1.0,0.9,0.999,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,64,128,"[0.25, 0.25, 0.25, 0.25]",100,1.0,0.9,0.999,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,64,128,"[0.25, 0.25, 0.25, 0.25]",256,1.0,0.9,0.999,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,64,128,"[0.25, 0.25, 0.25, 0.25]",100,1.0,0.9,0.999,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,64,128,"[0.25, 0.25, 0.25, 0.25]",153,1.0,0.9,0.999,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,64,128,"[0.25, 0.25, 0.25, 0.25]",153,1.0,0.9,0.999,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
experiments_dataframe

,DIM_img,DIM_text,alpha_modalities,batch_size,beta,beta_1,beta_2,beta_content,beta_m1_style,beta_m2_style,beta_m3_style,beta_style,calc_nll,calc_prd,class_dim,clf_save_m1,clf_save_m2,clf_save_m3,dataloader_workers,dataset,decoder_save_m1,decoder_save_m2,decoder_save_m3,device,dir_checkpoints,dir_clf,dir_cond_gen,dir_data,dir_experiment,dir_experiment_run,dir_fid,dir_gen_eval,dir_gen_eval_fid,dir_inference,dir_logs,dir_logs_clf,dir_plots,dir_random_samples,dir_swapping,div_weight_m1_content,div_weight_m2_content,div_weight_m3_content,div_weight_uniform_content,encoder_save_m1,encoder_save_m2,encoder_save_m3,end_epoch,eval_freq,eval_freq_fid,eval_lr,experiment_uid,factorized_representation,image_channels,img_size,inception_state_dict,initial_learning_rate,joint_elbo,len_sequence,likelihood_m1,likelihood_m2,likelihood_m3,load_saved,method,mm_vae_save,modality_jsd,modality_moe,modality_poe,num_features,num_samples_fid,num_training_samples_lr,poe_unimodal_elbos,save_figure,start_epoch,steps_per_training_epoch,str_experiment,style_lat_dim,style_pa_dim,style_text_dim,text_encoding,use_clf,use_toy_dataset,total_test_loss,total_epochs,experiment_duration,vocab_size,img_clf_type,alphabet,feature_extractor_img,n_crops,clf_loss,distributed,max_early_stopping_index,reduce_lr_on_plateau,start_early_stopping_epoch,mean_epoch_time,lr_eval_Lung Opacity_PA,lr_eval_Lung Opacity_Lateral,lr_eval_Lung Opacity_text,lr_eval_Lung Opacity_Lateral_PA,lr_eval_Lung Opacity_PA_text,lr_eval_Lung Opacity_Lateral_text,lr_eval_Lung Opacity_Lateral_PA_text,lr_eval_Pleural Effusion_PA,lr_eval_Pleural Effusion_Lateral,lr_eval_Pleural Effusion_text,lr_eval_Pleural Effusion_Lateral_PA,lr_eval_Pleural Effusion_PA_text,lr_eval_Pleural Effusion_Lateral_text,lr_eval_Pleural Effusion_Lateral_PA_text,lr_eval_Support Devices_PA,lr_eval_Support Devices_Lateral,lr_eval_Support Devices_text,lr_eval_Support Devices_Lateral_PA,lr_eval_Support Devices_PA_text,lr_eval_Support Devices_Lateral_text,lr_eval_Support Devices_Lateral_PA_text,only_text_modality,fixed_image_extractor,text_clf_type,gen_eval_cond_Lung Opacity_text_text,gen_eval_cond_Pleural Effusion_text_text,gen_eval_cond_Support Devices_text_text,gen_eval_random_Lung Opacity,gen_eval_random_Pleural Effusion,gen_eval_random_Support Devices,lhoods_text_text,lhoods_text_joint,gen_eval_cond_Lung Opacity_PA_PA,gen_eval_cond_Lung Opacity_PA_Lateral,gen_eval_cond_Lung Opacity_PA_text,gen_eval_cond_Lung Opacity_Lateral_PA,gen_eval_cond_Lung Opacity_Lateral_Lateral,gen_eval_cond_Lung Opacity_Lateral_text,gen_eval_cond_Lung Opacity_text_PA,gen_eval_cond_Lung Opacity_text_Lateral,gen_eval_cond_Lung Opacity_Lateral_PA_PA,gen_eval_cond_Lung Opacity_Lateral_PA_Lateral,gen_eval_cond_Lung Opacity_Lateral_PA_text,gen_eval_cond_Lung Opacity_PA_text_PA,gen_eval_cond_Lung Opacity_PA_text_Lateral,gen_eval_cond_Lung Opacity_PA_text_text,gen_eval_cond_Lung Opacity_Lateral_text_PA,gen_eval_cond_Lung Opacity_Lateral_text_Lateral,gen_eval_cond_Lung Opacity_Lateral_text_text,gen_eval_cond_Lung Opacity_Lateral_PA_text_PA,gen_eval_cond_Lung Opacity_Lateral_PA_text_Lateral,gen_eval_cond_Lung Opacity_Lateral_PA_text_text,lhoods_PA_PA,lhoods_PA_Lateral,lhoods_PA_text,lhoods_PA_joint,lhoods_Lateral_PA,lhoods_Lateral_Lateral,lhoods_Lateral_text,lhoods_Lateral_joint,lhoods_text_PA,lhoods_text_Lateral,lhoods_Lateral_PA_PA,lhoods_Lateral_PA_Lateral,lhoods_Lateral_PA_text,lhoods_Lateral_PA_joint,lhoods_PA_text_PA,lhoods_PA_text_Lateral,lhoods_PA_text_text,lhoods_PA_text_joint,lhoods_Lateral_text_PA,lhoods_Lateral_text_Lateral,lhoods_Lateral_text_text,lhoods_Lateral_text_joint,lhoods_Lateral_PA_text_PA,lhoods_Lateral_PA_text_Lateral,lhoods_Lateral_PA_text_text,lhoods_Lateral_PA_text_joint,gen_eval_cond_Pleural Effusion_PA_PA,gen_eval_cond_Pleural Effusion_PA_Lateral,gen_eval_cond_Pleural Effusion_PA_text,gen_eval_cond_Pleural Effusion_Lateral_PA,gen_eval_cond_Pleural Effusion_Lateral_Lateral,gen_eval_cond_Pleural Effusion_Lateral_text,gen_eval_cond_Pleural E

# Does a trimodal MMVAE produce better results than a Unimodal one?
### yes

In [70]:
unimodal_exp_df = experiments_dataframe.loc[experiments_dataframe['only_text_modality'] == True]
txt_lr_eval_columns = ['lr_eval_Lung Opacity_text', 'lr_eval_Pleural Effusion_text', 'lr_eval_Support Devices_text']
print(txt_lr_eval_columns)

['lr_eval_Lung Opacity_text', 'lr_eval_Pleural Effusion_text', 'lr_eval_Support Devices_text']


In [75]:
temp_exp_df = experiments_dataframe.dropna(subset= [*txt_lr_eval_columns, 'only_text_modality'])

temp1 = temp_exp_df.loc[(temp_exp_df['method'] == 'joint_elbo')&(temp_exp_df['only_text_modality'] == True), ['only_text_modality', 'text_encoding', 'total_test_loss', 'total_epochs', 'class_dim',*txt_lr_eval_columns ]].dropna(how='all', axis=1)
#df1 = temp1.loc[temp1['total_test_loss'] == temp1['total_test_loss'].min()]

temp2 = temp_exp_df.loc[(temp_exp_df['method'] == 'joint_elbo'), ['only_text_modality', 'total_test_loss','text_encoding',  'total_epochs', 'class_dim',*txt_lr_eval_columns ]].dropna(how='all', axis=1)
#df2 = temp2.loc[temp2['total_test_loss'] == temp2['total_test_loss'].min()]


display(temp1.append(temp2).set_index('only_text_modality').sort_values(by=txt_lr_eval_columns,ascending=False))

,text_encoding,total_test_loss,total_epochs,class_dim,lr_eval_Lung Opacity_text,lr_eval_Pleural Effusion_text,lr_eval_Support Devices_text
only_text_modality,,,,,,,
False,char,1.590275e+06,15,64,0.182523,0.092857,0.024762
False,char,1.398855e+06,82,64,0.147446,0.066015,0.022381
True,word,2.749916e+02,72,64,0.146012,0.060363,0.024687
True,word,2.749916e+02,72,64,0.146012,0.060363,0.024687
False,char,1.487811e+06,16,64,0.134330,0.061629,0.095714
False,char,1.404091e+06,69,64,0.126123,0.092946,0.022381
True,word,1.260738e+03,41,64,0.118684,0.087003,0.024687
True,word,1.260738e+03,41,64,0.118684,0.087003,0.024687
True,char,6.670386e+02,91,64,0.114048,0.077922,0.023571


# Does the densenet feature extractor produce better results than the resnet feature extractor with 128/256 img size?

In [77]:
temp_exp_df = experiments_dataframe.dropna(subset= lr_eval_columns)

temp1 = temp_exp_df.loc[(temp_exp_df['method'] == 'joint_elbo')&(temp_exp_df['feature_extractor_img'] == 'densenet'), ['feature_extractor_img', 'total_test_loss', 'img_size', 'total_epochs', 'class_dim',*eval_columns ]].dropna(how='all', axis=1)
df1 = temp1.loc[temp1['total_test_loss'] == temp1['total_test_loss'].min()]

temp2 = temp_exp_df.loc[(temp_exp_df['method'] == 'joint_elbo')&(temp_exp_df['feature_extractor_img'] == 'resnet'), ['feature_extractor_img', 'total_test_loss','img_size',  'total_epochs', 'class_dim',*eval_columns ]].dropna(how='all', axis=1)
df2 = temp2.loc[temp2['total_test_loss'] == temp2['total_test_loss'].min()]


display(temp1.append(temp2).set_index('feature_extractor_img').sort_values(by=lr_eval_columns,ascending=False))

,total_test_loss,img_size,total_epochs,class_dim,lr_eval_Lung Opacity_PA,lr_eval_Lung Opacity_Lateral,lr_eval_Lung Opacity_text,lr_eval_Lung Opacity_Lateral_PA,lr_eval_Lung Opacity_PA_text,lr_eval_Lung Opacity_Lateral_text,lr_eval_Lung Opacity_Lateral_PA_text,lr_eval_Pleural Effusion_PA,lr_eval_Pleural Effusion_Lateral,lr_eval_Pleural Effusion_text,lr_eval_Pleural Effusion_Lateral_PA,lr_eval_Pleural Effusion_PA_text,lr_eval_Pleural Effusion_Lateral_text,lr_eval_Pleural Effusion_Lateral_PA_text,lr_eval_Support Devices_PA,lr_eval_Support Devices_Lateral,lr_eval_Support Devices_text,lr_eval_Support Devices_Lateral_PA,lr_eval_Support Devices_PA_text,lr_eval_Support Devices_Lateral_text,lr_eval_Support Devices_Lateral_PA_text
feature_extractor_img,,,,,,,,,,,,,,,,,,,,,,,,,
resnet,6.357424e+04,256,22,64,0.136562,0.150969,0.105000,0.129326,0.114241,0.168111,0.138699,0.147103,0.069259,0.052500,0.097778,0.148889,0.069259,0.092569,0.025000,0.025000,0.025000,0.025000,0.025000,0.186667,0.025000
resnet,1.320720e+04,128,19,96,0.129939,0.108932,0.149186,0.111901,0.144425,0.113824,0.138840,0.066891,0.087473,0.079139,0.069880,0.070343,0.114597,0.106343,0.026144,0.026144,0.051743,0.026144,0.026144,0.026144,0.026144
densenet,1.398855e+06,256,82,64,0.129740,0.112525,0.147446,0.121067,0.166777,0.185475,0.145994,0.279479,0.100712,0.066015,0.101779,0.171134,0.118861,0.093092,0.085913,0.033214,0.022381,0.055238,0.022381,0.024881,0.022381
resnet,1.697511e+04,128,85,64,0.118837,0.131597,0.129948,0.120486,0.121496,0.202841,0.146007,0.092105,0.098937,0.074219,0.078102,0.087431,0.096335,0.087431,0.023438,0.023438,0.023438,0.023438,0.023438,0.023438,0.023438
densenet,1.404091e+06,256,69,64,0.117545,0.112912,0.126123,0.107181,0.139627,0.122013,0.153367,0.086284,0.113347,0.092946,0.114369,0.092711,0.146260,0.137437,0.022381,0.022381,0.022381,0.022381,0.022381,0.022381,0.026964
densenet,3.195840e+05,256,42,64,0.115423,0.149662,0.163697,0.122664,0.133038,0.221178,0.210575,0.136123,0.103317,0.180205,0.122964,0.125906,0.071299,0.162831,0.035675,0.023965,0.023965,0.023965,0.064052,0.023965,0.023965
resnet,1.669414e+04,128,57,64,0.115303,0.110076,0.105000,0.122727,0.113093,0.148636,0.122727,0.104563,0.111786,0.057500,0.142698,0.078690,0.145000,0.146270,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000
densenet,1.487811e+06,256,16,64,0.111414,0.160536,0.134330,0.125908,0.129253,0.173564,0.168936,0.082917,0.054524,0.061629,0.054524,0.059436,0.083991,0.062886,0.025952,0.025952,0.095714,0.031429,0.060000,0.034490,0.029780
densenet,1.590275e+06,256,15,64,0.104524,0.105293,0.182523,0.110116,0.110580,0.128576,0.126386,0.053333,0.060000,0.092857,0.074405,0.053333,0.070000,0.063333,0.024762,0.024762,0.024762,0.053155,0.024762,0.024762,0.068155


# What works best for the pretrained densenet feature extractor, fine-tuning or transfer learning?
### Transfer learning seems to give better results (to be tried again)

In [88]:
temp_exp_df = experiments_dataframe.dropna(subset= [*lr_eval_columns,'fixed_image_extractor'])
temp_exp_df.loc[temp_exp_df['fixed_image_extractor'] == False, 'total_test_loss']

Series([], Name: total_test_loss, dtype: float64)

In [89]:
temp_exp_df = experiments_dataframe
temp1 = temp_exp_df.loc[(temp_exp_df['fixed_image_extractor'] == True)&(temp_exp_df['method'] == 'joint_elbo')&(temp_exp_df['feature_extractor_img'] == 'densenet'), ['total_test_loss', 'total_epochs','fixed_image_extractor', 'class_dim',*lr_eval_columns ]].dropna(how='all', axis=1)
df1 = temp1.loc[temp1['total_test_loss'] == temp1['total_test_loss'].min()]

temp = temp_exp_df.loc[(temp_exp_df['fixed_image_extractor'] == False)&(temp_exp_df['method'] == 'joint_elbo')&(temp_exp_df['feature_extractor_img'] == 'densenet'), ['total_test_loss', 'total_epochs','fixed_image_extractor', 'class_dim',*lr_eval_columns ]].dropna(how='all', axis=1)
df2 = temp.loc[temp['total_test_loss'] == temp['total_test_loss'].min()]


display(df1.append(df2).reset_index().rename(index = {0:'transfer learning', 1:'fine tuning'}).sort_values(by=lr_eval_columns,ascending=False))
# display(temp1.append(temp2).set_index('fixed_image_extractor').sort_values(by=lr_eval_columns,ascending=False))


,index,total_test_loss,total_epochs,fixed_image_extractor,class_dim,lr_eval_Lung Opacity_PA,lr_eval_Lung Opacity_Lateral,lr_eval_Lung Opacity_text,lr_eval_Lung Opacity_Lateral_PA,lr_eval_Lung Opacity_PA_text,lr_eval_Lung Opacity_Lateral_text,lr_eval_Lung Opacity_Lateral_PA_text,lr_eval_Pleural Effusion_PA,lr_eval_Pleural Effusion_Lateral,lr_eval_Pleural Effusion_text,lr_eval_Pleural Effusion_Lateral_PA,lr_eval_Pleural Effusion_PA_text,lr_eval_Pleural Effusion_Lateral_text,lr_eval_Pleural Effusion_Lateral_PA_text,lr_eval_Support Devices_PA,lr_eval_Support Devices_Lateral,lr_eval_Support Devices_text,lr_eval_Support Devices_Lateral_PA,lr_eval_Support Devices_PA_text,lr_eval_Support Devices_Lateral_text,lr_eval_Support Devices_Lateral_PA_text
transfer learning,29,1.398855e+06,82,1.0,64,0.129740,0.112525,0.147446,0.121067,0.166777,0.185475,0.145994,0.279479,0.100712,0.066015,0.101779,0.171134,0.118861,0.093092,0.085913,0.033214,0.022381,0.055238,0.022381,0.024881,0.022381
fine tuning,32,1.765943e+06,28,0.0,64,0.121901,0.109887,0.137535,0.109887,0.109887,0.138859,0.120670,0.074667,0.111308,0.113426,0.109076,0.055883,0.071691,0.095500,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Does a larger class_dim improve the score?
### yes

In [92]:
temp_exp_df = experiments_dataframe.dropna(subset= lr_eval_columns)

temp1 = temp_exp_df.loc[(temp_exp_df['method'] == 'joint_elbo')&(temp_exp_df['class_dim'] == 64)&(experiments_dataframe['img_size'] == 128), ['feature_extractor_img','class_dim', 'total_test_loss', 'img_size', 'total_epochs', 'class_dim',*eval_columns ]].dropna(how='all', axis=1)
#df1 = temp1.loc[temp1['total_test_loss'] == temp1['total_test_loss'].min()]

temp2 = temp_exp_df.loc[(temp_exp_df['method'] == 'joint_elbo')&(temp_exp_df['class_dim'] != 64)&(experiments_dataframe['img_size'] == 128), ['feature_extractor_img','class_dim', 'total_test_loss', 'img_size', 'total_epochs', 'class_dim',*eval_columns ]].dropna(how='all', axis=1)
#df2 = temp2.loc[temp1['total_test_loss'] == temp1['total_test_loss'].min()]

In [94]:
display(temp1.append(temp2).sort_values(by=lr_eval_columns,ascending=False))

,feature_extractor_img,class_dim,total_test_loss,img_size,total_epochs,class_dim,lr_eval_Lung Opacity_PA,lr_eval_Lung Opacity_Lateral,lr_eval_Lung Opacity_text,lr_eval_Lung Opacity_Lateral_PA,lr_eval_Lung Opacity_PA_text,lr_eval_Lung Opacity_Lateral_text,lr_eval_Lung Opacity_Lateral_PA_text,lr_eval_Pleural Effusion_PA,lr_eval_Pleural Effusion_Lateral,lr_eval_Pleural Effusion_text,lr_eval_Pleural Effusion_Lateral_PA,lr_eval_Pleural Effusion_PA_text,lr_eval_Pleural Effusion_Lateral_text,lr_eval_Pleural Effusion_Lateral_PA_text,lr_eval_Support Devices_PA,lr_eval_Support Devices_Lateral,lr_eval_Support Devices_text,lr_eval_Support Devices_Lateral_PA,lr_eval_Support Devices_PA_text,lr_eval_Support Devices_Lateral_text,lr_eval_Support Devices_Lateral_PA_text
22,resnet,96,13207.20020,128,19,96,0.129939,0.108932,0.149186,0.111901,0.144425,0.113824,0.138840,0.066891,0.087473,0.079139,0.069880,0.070343,0.114597,0.106343,0.026144,0.026144,0.051743,0.026144,0.026144,0.026144,0.026144
0,NaN,64,16879.25586,128,299,64,0.127363,0.153607,0.119420,0.126877,0.154948,0.182021,0.158118,0.056276,0.060985,0.122210,0.062686,0.054688,0.114397,0.172519,0.027344,0.027344,0.027344,0.027344,0.027344,0.027344,0.027344
17,resnet,64,16975.10938,128,85,64,0.118837,0.131597,0.129948,0.120486,0.121496,0.202841,0.146007,0.092105,0.098937,0.074219,0.078102,0.087431,0.096335,0.087431,0.023438,0.023438,0.023438,0.023438,0.023438,0.023438,0.023438
9,resnet,64,16694.13818,128,57,64,0.115303,0.110076,0.105000,0.122727,0.113093,0.148636,0.122727,0.104563,0.111786,0.057500,0.142698,0.078690,0.145000,0.146270,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000


In [23]:
temp = experiments_dataframe.loc[(experiments_dataframe['class_dim'] == 64)&(experiments_dataframe['method'] == 'joint_elbo')&(experiments_dataframe['img_size'] == 128), ['total_test_loss', 'total_epochs', 'class_dim']]
display(temp.loc[temp['total_test_loss'] == temp['total_test_loss'].min()])

temp = experiments_dataframe.loc[(experiments_dataframe['class_dim'] != 64)&(experiments_dataframe['method'] == 'joint_elbo')&(experiments_dataframe['img_size'] == 128), ['total_test_loss', 'total_epochs', 'class_dim']]
display(temp.loc[temp['total_test_loss'] == temp['total_test_loss'].min()])

,total_test_loss,total_epochs,class_dim
31,274.991572,72,64


,total_test_loss,total_epochs,class_dim
22,13207.2002,19,96


# Does the word encoding improve the separability of the latent representations?

## For unimodal experiments:

In [103]:
temp_exp_df = experiments_dataframe.dropna(subset= txt_lr_eval_columns)

temp1 = temp_exp_df.loc[(experiments_dataframe['text_encoding'] == 'word')&(experiments_dataframe['method'] == 'joint_elbo')&(experiments_dataframe['only_text_modality'] == True), ['feature_extractor_img','text_encoding', 'total_test_loss', 'img_size', 'total_epochs', 'class_dim',*eval_columns ]].dropna(how='all', axis=1)
#df1 = temp1.loc[temp1['total_test_loss'] == temp1['total_test_loss'].min()]


temp2 = temp_exp_df.loc[(experiments_dataframe['text_encoding'] == 'char')&(experiments_dataframe['method'] == 'joint_elbo')&(experiments_dataframe['only_text_modality'] == True), ['feature_extractor_img','text_encoding', 'total_test_loss', 'img_size', 'total_epochs', 'class_dim',*eval_columns ]].dropna(how='all', axis=1)
#df2 = temp2.loc[temp2['total_test_loss'] == temp2['total_test_loss'].min()]


display(temp1.append(temp2).set_index('text_encoding').sort_values(by=txt_lr_eval_columns,ascending=False))

,feature_extractor_img,total_test_loss,img_size,total_epochs,class_dim,lr_eval_Lung Opacity_text,lr_eval_Pleural Effusion_text,lr_eval_Support Devices_text,gen_eval_cond_Lung Opacity_text_text,gen_eval_cond_Pleural Effusion_text_text,gen_eval_cond_Support Devices_text_text,gen_eval_random_Lung Opacity,gen_eval_random_Pleural Effusion,gen_eval_random_Support Devices,lhoods_text_text,lhoods_text_joint
text_encoding,,,,,,,,,,,,,,,,
word,resnet,274.991572,128,72,64,0.146012,0.060363,0.024687,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
word,resnet,1260.738464,128,41,64,0.118684,0.087003,0.024687,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
char,resnet,667.038559,128,91,64,0.114048,0.077922,0.023571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
char,resnet,2391.336853,128,10,64,0.113968,0.072185,0.022381,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
char,resnet,1658.064178,128,26,64,0.110105,0.068730,0.022381,0.095177,0.049205,0.028411,1.0,1.0,1.0,-1209.635864,-1209.635864


## For multimodal experiments:

In [104]:
temp_exp_df = experiments_dataframe.dropna(subset= lr_eval_columns)

temp1 = temp_exp_df.loc[(experiments_dataframe['text_encoding'] == 'word')&(experiments_dataframe['method'] == 'joint_elbo'), ['feature_extractor_img','text_encoding', 'total_test_loss', 'img_size', 'total_epochs', 'class_dim',*eval_columns ]].dropna(how='all', axis=1)
#df1 = temp1.loc[temp1['total_test_loss'] == temp1['total_test_loss'].min()]


temp2 = temp_exp_df.loc[(experiments_dataframe['text_encoding'] == 'char')&(experiments_dataframe['method'] == 'joint_elbo'), ['feature_extractor_img','text_encoding', 'total_test_loss', 'img_size', 'total_epochs', 'class_dim',*eval_columns ]].dropna(how='all', axis=1)
#df2 = temp2.loc[temp2['total_test_loss'] == temp2['total_test_loss'].min()]


display(temp1.append(temp2).set_index('text_encoding').sort_values(by=lr_eval_columns,ascending=False))

,feature_extractor_img,total_test_loss,img_size,total_epochs,class_dim,lr_eval_Lung Opacity_PA,lr_eval_Lung Opacity_Lateral,lr_eval_Lung Opacity_text,lr_eval_Lung Opacity_Lateral_PA,lr_eval_Lung Opacity_PA_text,lr_eval_Lung Opacity_Lateral_text,lr_eval_Lung Opacity_Lateral_PA_text,lr_eval_Pleural Effusion_PA,lr_eval_Pleural Effusion_Lateral,lr_eval_Pleural Effusion_text,lr_eval_Pleural Effusion_Lateral_PA,lr_eval_Pleural Effusion_PA_text,lr_eval_Pleural Effusion_Lateral_text,lr_eval_Pleural Effusion_Lateral_PA_text,lr_eval_Support Devices_PA,lr_eval_Support Devices_Lateral,lr_eval_Support Devices_text,lr_eval_Support Devices_Lateral_PA,lr_eval_Support Devices_PA_text,lr_eval_Support Devices_Lateral_text,lr_eval_Support Devices_Lateral_PA_text
text_encoding,,,,,,,,,,,,,,,,,,,,,,,,,,
word,NaN,6.697227e+04,256,99,64,0.155720,0.131389,0.134706,0.153046,0.189580,0.158792,0.167623,0.071759,0.114028,0.065000,0.126256,0.071759,0.132361,0.101979,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000
word,NaN,6.728372e+04,256,18,64,0.136944,0.124882,0.132500,0.133950,0.113910,0.135085,0.133681,0.230278,0.115000,0.052500,0.151319,0.167500,0.133767,0.137153,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000
word,resnet,6.357424e+04,256,22,64,0.136562,0.150969,0.105000,0.129326,0.114241,0.168111,0.138699,0.147103,0.069259,0.052500,0.097778,0.148889,0.069259,0.092569,0.025000,0.025000,0.025000,0.025000,0.025000,0.186667,0.025000
word,resnet,1.320720e+04,128,19,96,0.129939,0.108932,0.149186,0.111901,0.144425,0.113824,0.138840,0.066891,0.087473,0.079139,0.069880,0.070343,0.114597,0.106343,0.026144,0.026144,0.051743,0.026144,0.026144,0.026144,0.026144
char,densenet,1.398855e+06,256,82,64,0.129740,0.112525,0.147446,0.121067,0.166777,0.185475,0.145994,0.279479,0.100712,0.066015,0.101779,0.171134,0.118861,0.093092,0.085913,0.033214,0.022381,0.055238,0.022381,0.024881,0.022381
char,NaN,1.687926e+04,128,299,64,0.127363,0.153607,0.119420,0.126877,0.154948,0.182021,0.158118,0.056276,0.060985,0.122210,0.062686,0.054688,0.114397,0.172519,0.027344,0.027344,0.027344,0.027344,0.027344,0.027344,0.027344
char,resnet,1.697511e+04,128,85,64,0.118837,0.131597,0.129948,0.120486,0.121496,0.202841,0.146007,0.092105,0.098937,0.074219,0.078102,0.087431,0.096335,0.087431,0.023438,0.023438,0.023438,0.023438,0.023438,0.023438,0.023438
char,densenet,1.404091e+06,256,69,64,0.117545,0.112912,0.126123,0.107181,0.139627,0.122013,0.153367,0.086284,0.113347,0.092946,0.114369,0.092711,0.146260,0.137437,0.022381,0.022381,0.022381,0.022381,0.022381,0.022381,0.026964
word,densenet,3.195840e+05,256,42,64,0.115423,0.149662,0.163697,0.122664,0.133038,0.221178,0.210575,0.136123,0.103317,0.180205,0.122964,0.125906,0.071299,0.162831,0.035675,0.023965,0.023965,0.023965,0.064052,0.023965,0.023965


# Test of the latent representations of the MMVAE
### Average precision (AP) scores on the latent representations, averaged over the batches.
### AP summarizes a precision-recall curve as the weighted mean of precisions achieved at each threshold, with the increase in recall from the previous threshold used as the weight.
### Note: For medical data, the true negative rate (recall) is probably more important.

## The labels are not evenly distributed over the datasets, this needs to be taken into account when analyzing classification scores.

In [5]:
alphabet_path = os.path.join(str(Path(os.getcwd())), 'alphabet.json')
with open(alphabet_path) as alphabet_file:
    alphabet = str(''.join(json.load(alphabet_file)))

FLAGS = parser.parse_args([])

config_path = get_config_path()
with open(config_path, 'rt') as json_file:
    t_args = argparse.Namespace()
    t_args.__dict__.update(json.load(json_file))
    FLAGS = parser.parse_args('', namespace=t_args)
    
FLAGS = expand_paths(FLAGS)
FLAGS.dir_data = os.path.expanduser(FLAGS.dir_data)
dir_data = FLAGS.dir_data
dir_dataset = os.path.join(dir_data, 'files_small_128')

In [6]:
labels = ['Lung Opacity', 'Pleural Effusion', 'Support Devices']
test_labels_path = os.path.join(dir_dataset, 'eval_labels.csv')
test_labels_df = pd.read_csv(test_labels_path)[labels].fillna(0)


In [7]:
counts = test_labels_df[test_labels_df == 1].count()
print(counts)

Lung Opacity        54
Pleural Effusion    29
Support Devices     12
dtype: int64


In [8]:
FLAGS.num_features = len(alphabet)
FLAGS.batch_size = 300
FLAGS.str_experiment = 'temp'
FLAGS.device = 'cuda'
FLAGS.dir_gen_eval_fid = ''
FLAGS.alpha_modalities = [FLAGS.div_weight_uniform_content, FLAGS.div_weight_m1_content,
                          FLAGS.div_weight_m2_content, FLAGS.div_weight_m3_content]

## A random classifier that classifies with a "most frequent" strategy, would have a mean average precision of 0.07

In [9]:
FLAGS.img_size = 128
average_precisions = test_dummy(FLAGS, alphabet)
print(np.mean(average_precisions))

setting dataset
setting modalities
setting clfs
setting rec_weights
dict_keys(['real', 'random', '', 'PA', 'Lateral', 'text', 'Lateral_PA', 'PA_text', 'Lateral_text', 'Lateral_PA_text'])
0.056666666666666664


In [10]:
def get_lr_eval_dict(path):
    lr_eval = {}
    for lr in os.listdir(os.path.join(lr_eval_dir, label)):
                            lr_eval[label][lr] = dict()
                            for logfile in os.listdir(os.path.join(lr_eval_dir, label, lr)):
                                for summary in summary_iterator(os.path.join(lr_eval_dir, label, lr, logfile)):
                                    value = summary.summary.value
                                    temp = str(value).split('\n')
                                    for elem in temp:
                                        elem = elem
                                        if elem.startswith('simple_value'):
                                            lr_eval[label][lr][summary.step] = elem.split(' ')[1]
    return 

In [36]:
checkpoint_path = os.path.expanduser(FLAGS.dir_fid)
lr_evals = dict()
for modality_method in ['moe']:
    for factorization in os.listdir(os.path.join(checkpoint_path, modality_method)):
        for experiment in os.listdir(os.path.join(checkpoint_path, modality_method, factorization)):
            if experiment.startswith('Mimic'):
                # print(experiments_dataframe.loc[experiments_dataframe['experiment_uid'] == experiment])
                lr_evals[experiment] = dict()
                lr_eval = lr_evals[experiment]
                experiment_dir_ = os.path.join(checkpoint_path, modality_method, factorization, experiment)
                lr_eval_dir = os.path.join(experiment_dir_, 'logs', 'Latent Representation')
                if os.path.exists(lr_eval_dir):
                    for label in os.listdir(lr_eval_dir):
                        lr_eval[label] = dict()
                        for lr in os.listdir(os.path.join(lr_eval_dir, label)):
                            lr_eval[label][lr] = dict()
                            for logfile in os.listdir(os.path.join(lr_eval_dir, label, lr)):
                                for summary in summary_iterator(os.path.join(lr_eval_dir, label, lr, logfile)):
                                    value = summary.summary.value
                                    temp = str(value).split('\n')
                                    for elem in temp:
                                        elem = elem
                                        if elem.startswith('simple_value'):
                                            lr_eval[label][lr][summary.step] = elem.split(' ')[1]


                        

In [37]:
import itertools
lrs = ['PA_text', 'Lateral_PA_text', 'PA', 'text', 'Lateral_text', 'Lateral_PA', 'Lateral']
for modality_method in ['moe']:
    for factorization in os.listdir(os.path.join(checkpoint_path, modality_method)):
        for experiment in os.listdir(os.path.join(checkpoint_path, modality_method, factorization)):
            if experiment.startswith('Mimic') and not os.path.exists(os.path.join(checkpoint_path, modality_method, factorization, experiment,'logs', 'Latent Representation')):
                lr_evals[experiment] = {}
                for label in labels:
                    lr_evals[experiment][label] = {}
                    for lr in lrs:
                        lr_evals[experiment][label][lr] = {}
                lr_eval = lr_evals[experiment]
                experiment_dir_ = os.path.join(checkpoint_path, modality_method, factorization, experiment)
                if not 'Latent Representation' in os.listdir(os.path.join(experiment_dir_, 'logs')):
                    for label , lr in itertools.product(labels, lrs):
                        logdir = os.path.join(experiment_dir_, 'logs', f'Latent Representation_{label}_{lr}')
                        if os.path.exists(logdir):
                            for logfile in os.listdir(logdir):
                                for summary in summary_iterator(os.path.join(logdir, logfile)):
                                    value = summary.summary.value
                                    temp = str(value).split('\n')
                                    for elem in temp:
                                        elem = elem
                                        if elem.startswith('simple_value'):
                                            lr_eval[label][lr][summary.step] = elem.split(' ')[1]
                

In [38]:
experiments_dataframe = pd.read_csv('experiments_dataframe.csv')
dfs = []
for experiment in lr_evals.keys():
    experiment_evals = lr_evals[experiment]
    if experiment_evals:
        for label in experiment_evals.keys():
            for lr in experiment_evals[label].keys():
                steps = experiment_evals[label][lr].keys()
                if steps:
                    max_step = max(steps) 
                    experiment_evals[label][lr] = experiment_evals[label][lr][max_step]
                else:
                    experiment_evals[label][lr] = None
                    
        flattened_experiment_evals = utils.flatten({'lr_eval':experiment_evals})
        for key in flattened_experiment_evals:
            experiments_dataframe.loc[
                    experiments_dataframe['experiment_uid'] == experiment, key] = flattened_experiment_evals[key]

        
        df = pd.DataFrame(experiment_evals).astype(float)
        index = df.index
        index.name = f'Steps: {max_step}'
        df['mean_AP'] = df.mean(numeric_only=True, axis=1)
        dfs.append((df, experiment))
experiments_dataframe.to_csv('experiments_dataframe.csv', index = False)

,DIM_img,DIM_text,alpha_modalities,batch_size,beta,beta_1,beta_2,beta_content,beta_m1_style,beta_m2_style,...,lr_eval_Support Devices_PA,lr_eval_Support Devices_Lateral,lr_eval_Support Devices_text,lr_eval_Support Devices_Lateral_PA,lr_eval_Support Devices_PA_text,lr_eval_Support Devices_Lateral_text,lr_eval_Support Devices_Lateral_PA_text,only_text_modality,fixed_image_extractor,config_path


,DIM_img,DIM_text,alpha_modalities,batch_size,beta,beta_1,beta_2,beta_content,beta_m1_style,beta_m2_style,...,lr_eval_Support Devices_PA,lr_eval_Support Devices_Lateral,lr_eval_Support Devices_text,lr_eval_Support Devices_Lateral_PA,lr_eval_Support Devices_PA_text,lr_eval_Support Devices_Lateral_text,lr_eval_Support Devices_Lateral_PA_text,only_text_modality,fixed_image_extractor,config_path


,DIM_img,DIM_text,alpha_modalities,batch_size,beta,beta_1,beta_2,beta_content,beta_m1_style,beta_m2_style,...,lr_eval_Support Devices_PA,lr_eval_Support Devices_Lateral,lr_eval_Support Devices_text,lr_eval_Support Devices_Lateral_PA,lr_eval_Support Devices_PA_text,lr_eval_Support Devices_Lateral_text,lr_eval_Support Devices_Lateral_PA_text,only_text_modality,fixed_image_extractor,config_path
21,64,128,"[0.25, 0.25, 0.25, 0.25]",153,1.0,0.9,0.999,1.0,1.0,1.0,...,0.03567538037896156,0.0239651408046484,0.0239651408046484,0.0239651408046484,0.0640522912144661,0.0239651408046484,0.0239651408046484,NaN,NaN,NaN


,DIM_img,DIM_text,alpha_modalities,batch_size,beta,beta_1,beta_2,beta_content,beta_m1_style,beta_m2_style,...,lr_eval_Support Devices_PA,lr_eval_Support Devices_Lateral,lr_eval_Support Devices_text,lr_eval_Support Devices_Lateral_PA,lr_eval_Support Devices_PA_text,lr_eval_Support Devices_Lateral_text,lr_eval_Support Devices_Lateral_PA_text,only_text_modality,fixed_image_extractor,config_path
24,64,128,"[0.25, 0.25, 0.25, 0.25]",300,1.0,0.9,0.999,1.0,1.0,1.0,...,None,None,0.022380951792001724,None,None,None,None,True,NaN,NaN


,DIM_img,DIM_text,alpha_modalities,batch_size,beta,beta_1,beta_2,beta_content,beta_m1_style,beta_m2_style,...,lr_eval_Support Devices_PA,lr_eval_Support Devices_Lateral,lr_eval_Support Devices_text,lr_eval_Support Devices_Lateral_PA,lr_eval_Support Devices_PA_text,lr_eval_Support Devices_Lateral_text,lr_eval_Support Devices_Lateral_PA_text,only_text_modality,fixed_image_extractor,config_path
31,64,128,"[0.25, 0.25, 0.25, 0.25]",192,1.0,0.9,0.999,1.0,1.0,1.0,...,None,None,0.02468711882829666,None,None,None,None,True,1.0,NaN


...

,DIM_img,DIM_text,alpha_modalities,batch_size,beta,beta_1,beta_2,beta_content,beta_m1_style,beta_m2_style,...,lr_eval_Support Devices_PA,lr_eval_Support Devices_Lateral,lr_eval_Support Devices_text,lr_eval_Support Devices_Lateral_PA,lr_eval_Support Devices_PA_text,lr_eval_Support Devices_Lateral_text,lr_eval_Support Devices_Lateral_PA_text,only_text_modality,fixed_image_extractor,config_path
30,64,128,"[0.25, 0.25, 0.25, 0.25]",300,1.0,0.9,0.999,1.0,1.0,1.0,...,0.022380951792001724,0.022380951792001724,0.022380951792001724,0.022380951792001724,0.022380951792001724,0.022380951792001724,0.026964286342263222,False,1.0,NaN


,DIM_img,DIM_text,alpha_modalities,batch_size,beta,beta_1,beta_2,beta_content,beta_m1_style,beta_m2_style,...,lr_eval_Support Devices_PA,lr_eval_Support Devices_Lateral,lr_eval_Support Devices_text,lr_eval_Support Devices_Lateral_PA,lr_eval_Support Devices_PA_text,lr_eval_Support Devices_Lateral_text,lr_eval_Support Devices_Lateral_PA_text,only_text_modality,fixed_image_extractor,config_path
19,64,128,"[0.25, 0.25, 0.25, 0.25]",97,1.0,0.9,0.999,1.0,1.0,1.0,...,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN


,DIM_img,DIM_text,alpha_modalities,batch_size,beta,beta_1,beta_2,beta_content,beta_m1_style,beta_m2_style,...,lr_eval_Support Devices_PA,lr_eval_Support Devices_Lateral,lr_eval_Support Devices_text,lr_eval_Support Devices_Lateral_PA,lr_eval_Support Devices_PA_text,lr_eval_Support Devices_Lateral_text,lr_eval_Support Devices_Lateral_PA_text,only_text_modality,fixed_image_extractor,config_path
23,64,128,"[0.25, 0.25, 0.25, 0.25]",500,1.0,0.9,0.999,1.0,1.0,1.0,...,None,None,nan,None,None,None,None,True,NaN,NaN


,DIM_img,DIM_text,alpha_modalities,batch_size,beta,beta_1,beta_2,beta_content,beta_m1_style,beta_m2_style,...,lr_eval_Support Devices_PA,lr_eval_Support Devices_Lateral,lr_eval_Support Devices_text,lr_eval_Support Devices_Lateral_PA,lr_eval_Support Devices_PA_text,lr_eval_Support Devices_Lateral_text,lr_eval_Support Devices_Lateral_PA_text,only_text_modality,fixed_image_extractor,config_path
29,64,128,"[0.25, 0.25, 0.25, 0.25]",300,1.0,0.9,0.999,1.0,1.0,1.0,...,0.08591269701719284,0.03321428596973419,0.022380951792001724,0.05523809418082237,0.022380951792001724,0.02488095313310623,0.022380951792001724,False,1.0,NaN


,DIM_img,DIM_text,alpha_modalities,batch_size,beta,beta_1,beta_2,beta_content,beta_m1_style,beta_m2_style,...,lr_eval_Support Devices_PA,lr_eval_Support Devices_Lateral,lr_eval_Support Devices_text,lr_eval_Support Devices_Lateral_PA,lr_eval_Support Devices_PA_text,lr_eval_Support Devices_Lateral_text,lr_eval_Support Devices_Lateral_PA_text,only_text_modality,fixed_image_extractor,config_path


In [31]:
for df, experiment in dfs:
    if len(experiments_dataframe.loc[experiments_dataframe['experiment_uid'] == experiment]) == 1:
        flags = experiments_dataframe.loc[experiments_dataframe['experiment_uid'] == experiment]

        print(f'Experiment {experiment} with text encoding: {flags.text_encoding.item()}, '
              f'image size: {flags.img_size.item()}, method: {flags.method.item()} \n and trained '
              f'for {flags.total_epochs.item()} epochs with batch size: {flags.batch_size.item()} '
              f'and {flags.steps_per_training_epoch.item()} steps per training epoch')
        display(df)

Experiment Mimic_2020_11_23_10_28_33_804954 with text encoding: word, image size: 256, method: joint_elbo 
 and trained for 42 epochs with batch size: 153 and -1 steps per training epoch


,Lung Opacity,Pleural Effusion,Support Devices,mean_AP
Steps: 16000,,,,
PA_text,0.133038,0.125906,0.064052,0.107666
Lateral_PA_text,0.210575,0.162831,0.023965,0.132457
PA,0.115423,0.136123,0.035675,0.095740
text,0.163697,0.180205,0.023965,0.122622
Lateral_text,0.221178,0.071299,0.023965,0.105481
Lateral_PA,0.122664,0.122964,0.023965,0.089864
Lateral,0.149662,0.103317,0.023965,0.092315


Experiment Mimic_2020_11_26_14_08_27_779062 with text encoding: char, image size: 128, method: joint_elbo 
 and trained for 10 epochs with batch size: 300 and 5 steps per training epoch


,Lung Opacity,Pleural Effusion,Support Devices,mean_AP
Steps: 70,,,,
PA_text,NaN,NaN,NaN,NaN
Lateral_PA_text,NaN,NaN,NaN,NaN
PA,NaN,NaN,NaN,NaN
text,0.113968,0.072185,0.022381,0.069511
Lateral_text,NaN,NaN,NaN,NaN
Lateral_PA,NaN,NaN,NaN,NaN
Lateral,NaN,NaN,NaN,NaN


Experiment Mimic_2020_11_27_15_29_46_831953 with text encoding: word, image size: 128, method: joint_elbo 
 and trained for 72 epochs with batch size: 192 and -1 steps per training epoch


...

,Pleural Effusion,Support Devices,Lung Opacity,mean_AP
Steps: 12580,,,,
PA_text,0.121939,NaN,0.127862,0.124901
Lateral_PA_text,0.077642,NaN,0.152179,0.114910
PA,0.103683,NaN,0.111352,0.107518
text,0.141037,NaN,0.134153,0.137595
Lateral_text,0.187178,NaN,0.189335,0.188256
Lateral_PA,0.074517,NaN,0.141465,0.107991
Lateral,0.082850,NaN,0.135669,0.109259


Experiment Mimic_2020_11_26_10_52_57_715603 with text encoding: char, image size: 128, method: joint_elbo 
 and trained for 83 epochs with batch size: 500 and -1 steps per training epoch


,Lung Opacity,Pleural Effusion,Support Devices,mean_AP
Steps: 9760,,,,
PA_text,NaN,NaN,NaN,NaN
Lateral_PA_text,NaN,NaN,NaN,NaN
PA,NaN,NaN,NaN,NaN
text,NaN,NaN,NaN,NaN
Lateral_text,NaN,NaN,NaN,NaN
Lateral_PA,NaN,NaN,NaN,NaN
Lateral,NaN,NaN,NaN,NaN


Experiment Mimic_2020_11_27_03_05_47_020992 with text encoding: char, image size: 256, method: joint_elbo 
 and trained for 82 epochs with batch size: 300 and -1 steps per training epoch


,Lung Opacity,Pleural Effusion,Support Devices,mean_AP
Steps: 16320,,,,
PA_text,0.166777,0.171134,0.022381,0.120097
Lateral_PA_text,0.145994,0.093092,0.022381,0.087156
PA,0.129740,0.279479,0.085913,0.165044
text,0.147446,0.066015,0.022381,0.078614
Lateral_text,0.185475,0.118861,0.024881,0.109739
Lateral_PA,0.121067,0.101779,0.055238,0.092695
Lateral,0.112525,0.100712,0.033214,0.082151


In [98]:
df.index

Index(['PA_text', 'Lateral_PA_text', 'PA', 'text', 'Lateral_text',
       'Lateral_PA', 'Lateral'],
      dtype='object', name='Steps: 12580')

 # Evaluation of the classifiers
 All classifiers were trained for 100 epochs

In [11]:
labels = ['Lung Opacity', 'Pleural Effusion', 'Support Devices']
FLAGS.alphabet = alphabet
FLAGS.use_clf = True

## Evaluation of the character encoding and image size 128

In [12]:
results = test_clfs(FLAGS, 128, 'char')
for modality in results.keys():
    print(f'mean precision for {modality} classifier: ', np.mean(results[modality]['list_precision_vals']))

setting dataset
setting modalities
setting clfs
setting rec_weights
dict_keys(['real', 'random', '', 'PA', 'Lateral', 'text', 'Lateral_PA', 'PA_text', 'Lateral_text', 'Lateral_PA_text'])


/cluster/home/klugh/software/anaconda/envs/temp/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


mean precision for PA classifier:  0.23372465466391026
mean precision for Lateral classifier:  0.20999513514889828
mean precision for text classifier:  0.5551593504487973


In [13]:
for modality in results.keys():
    print(f'report {modality}: \n', results[modality]['report'])

report PA: 
                   precision    recall  f1-score   support

    Lung Opacity     0.0000    0.0000    0.0000        52
            None     0.8253    1.0000    0.9043       392
Pleural Effusion     0.0000    0.0000    0.0000        23
 Support Devices     0.0000    0.0000    0.0000         8

        accuracy                         0.8253       475
       macro avg     0.2063    0.2500    0.2261       475
    weighted avg     0.6811    0.8253    0.7463       475

report Lateral: 
                   precision    recall  f1-score   support

    Lung Opacity     0.0000    0.0000    0.0000        52
            None     0.8245    0.9949    0.9017       392
Pleural Effusion     0.0000    0.0000    0.0000        23
 Support Devices     0.0000    0.0000    0.0000         8

        accuracy                         0.8211       475
       macro avg     0.2061    0.2487    0.2254       475
    weighted avg     0.6804    0.8211    0.7442       475

report text: 
                   pr

## Evaluation of the word encoding and image size 128
### The text classifier precision is slightly better for the word encoding

In [14]:
results = test_clfs(FLAGS, 128, 'word', alphabet)
for modality in results.keys():
    print(f'mean precision for {modality} classifier: ', np.mean(results[modality]['list_precision_vals']))

setting dataset
setting modalities
setting clfs
setting rec_weights
dict_keys(['real', 'random', '', 'PA', 'Lateral', 'text', 'Lateral_PA', 'PA_text', 'Lateral_text', 'Lateral_PA_text'])


/cluster/home/klugh/software/anaconda/envs/temp/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


mean precision for PA classifier:  0.2619510306245376
mean precision for Lateral classifier:  0.2139644830777028
mean precision for text classifier:  0.5821441968107942


In [15]:
for modality in results.keys():
    print(f'report {modality}: \n', results[modality]['report'])

report PA: 
                   precision    recall  f1-score   support

    Lung Opacity     0.0000    0.0000    0.0000        52
            None     0.8253    1.0000    0.9043       392
Pleural Effusion     0.0000    0.0000    0.0000        23
 Support Devices     0.0000    0.0000    0.0000         8

        accuracy                         0.8253       475
       macro avg     0.2063    0.2500    0.2261       475
    weighted avg     0.6811    0.8253    0.7463       475

report Lateral: 
                   precision    recall  f1-score   support

    Lung Opacity     0.0000    0.0000    0.0000        52
            None     0.8245    0.9949    0.9017       392
Pleural Effusion     0.0000    0.0000    0.0000        23
 Support Devices     0.0000    0.0000    0.0000         8

        accuracy                         0.8211       475
       macro avg     0.2061    0.2487    0.2254       475
    weighted avg     0.6804    0.8211    0.7442       475

report text: 
                   pr

## Evaluation of image size 256


In [16]:

FLAGS.text_encoding = 'word'
FLAGS.img_size = 256
mimic_experiment = MimicExperiment(flags=FLAGS)
mimic_test = Mimic(FLAGS, mimic_experiment.labels,split='eval')

setting dataset
setting modalities
setting clfs
setting rec_weights
dict_keys(['real', 'random', '', 'PA', 'Lateral', 'text', 'Lateral_PA', 'PA_text', 'Lateral_text', 'Lateral_PA_text'])


In [17]:
report_pa, list_precision_pa = test_clf(FLAGS, mimic_experiment, mimic_test, 'PA')
report_lat, list_precision_lat = test_clf(FLAGS, mimic_experiment, mimic_test, 'Lateral')
report_text, list_precision_text = test_clf(FLAGS, mimic_experiment, mimic_test, 'text')
print('mean precision for pa classifier: ',np.mean(list_precision_pa))
print('mean precision for lat classifier: ',np.mean(list_precision_lat))
print('mean precision for text classifier: ',np.mean(list_precision_text))
print('report pa: ', report_pa)
print('report lat: ', report_lat)
print('report text: ', report_text)

RuntimeError: CUDA out of memory. Tried to allocate 2.34 GiB (GPU 0; 7.93 GiB total capacity; 5.45 GiB already allocated; 1.82 GiB free; 5.56 GiB reserved in total by PyTorch)

# Evaluation of densenet classifiers

In [18]:
# need to load mimic_test again  with clf type densenet
FLAGS.img_clf_type = 'densenet'
FLAGS.batch_size = 5
mimic_test = Mimic(FLAGS, labels, split='eval')

In [19]:
model = CheXNet(len(labels)).cuda()

In [20]:
checkpoint_path = os.path.expanduser('~/klugh/mimic/trained_classifiers_new/Mimic256_densenet/old_clf_m1_45')
checkpoint = torch.load(checkpoint_path)
checkpoint_keys = [*checkpoint.keys()]
for key in checkpoint_keys:
    checkpoint['.'.join(key.split('.')[1:])] = checkpoint[key]
    del checkpoint[key]
    
model.load_state_dict(checkpoint)
report, avg_precision = test_model(flags=FLAGS, model=model, clf_type='densenet', test_set= mimic_test, modality='PA', labels = labels)
print('average precision: ',avg_precision)
print(report)

FileNotFoundError: [Errno 2] No such file or directory: '/cluster/home/klugh/klugh/mimic/trained_classifiers_new/Mimic256_densenet/old_clf_m1_45'

In [ ]:
checkpoint_path = os.path.expanduser('~/klugh/mimic/trained_classifiers_new/Mimic256_densenet/old_clf_m2_43')
checkpoint = torch.load(checkpoint_path)
checkpoint_keys = [*checkpoint.keys()]
for key in checkpoint_keys:
    checkpoint['.'.join(key.split('.')[1:])] = checkpoint[key]
    del checkpoint[key]
    
model.load_state_dict(checkpoint)
report, avg_precision = test_model(flags=FLAGS, model=model, clf_type='densenet', test_set= mimic_test, modality='PA', labels = labels)
print('average precision: ',avg_precision)
print(report)